## Example Selector と Few-Shot Prompt

In [1]:
examples = [
    {
        "input": "長崎県長崎市のカフェを地図に表示してください。",
        "output": """[out:json][timeout:30000];
area["name"="長崎市"]->.searchArea;
(
  nwr["amenity"="cafe"](area.searchArea);
);
out geom;"""
    },
    {
        "input": "東京都台東区のラーメン屋を地図に表示してください。",
        "output": """[out:json][timeout:30000];
area["name"="台東区"]->.searchArea;
(
  nwr["amenity"="restaurant"]["cuisine"="ramen"](area.searchArea);
);
out geom;"""
    },
    {
        "input": "東京都渋谷区の駅を地図に表示してください。",
        "output": """[out:json][timeout:30000];
area["name"="渋谷区"]->.searchArea;
(
  nwr["railway"="station"](area.searchArea);
);
out geom;"""
    },
    {
        "input": "東京都新宿区の病院を地図に表示してください。",
        "output": """[out:json][timeout:30000];
area["name"="新宿区"]->.searchArea;
(
  nwr["amenity"="hospital"](area.searchArea);
);
out geom;"""
    },
    {
        "input": "大阪府大阪市のコンビニを地図に表示してください。",
        "output": """[out:json][timeout:30000];
area["name"="大阪市"]->.searchArea;
(
  nwr["amenity"="convenience"](area.searchArea);
);
out geom;"""
    },
]

In [2]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="""Input:
{input}

Output:
{output}""",
)

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [4]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=3,
)

In [5]:
from langchain_core.prompts import FewShotPromptTemplate

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="""You are an expert in OpenStreetMap and Overpass QL.
Output the appropriate Overpass QL from the user input.

You will always reply according to the following rules:
- Output valid Overpass QL.
- The query MUST be out json.
- The query MUST be out geom.
- The query MUST be set timeout as 30000.
- The query will utilize a area specifier as needed.
- The query will search nwr as needed.
- The query MUST be line delimited and surrounded by just three backquote to indicate that it is a code block.

** Examples: **
""",
    suffix="""User input:
{input}

Output:""",
    input_variables=["input"],
)

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

# モデルの準備
model = ChatGoogleGenerativeAI(model="gemini-exp-1206", temperature=0)

In [7]:
input_text = "東京都台東区のカフェを地図に表示してください。"

In [8]:
chain = similar_prompt | model

res = chain.invoke({"input": input_text})
result = res.content.strip()
print(result)

```
[out:json][timeout:30000];
area["name"="台東区"]->.searchArea;
(
  nwr["amenity"="cafe"](area.searchArea);
);
out geom;
```
